In [1]:
import pandas as pd

from lib.utils import get_nyse_date_tups, get_oxford_dfs

In [2]:
start, end = '2018-10-11', '2023-10-09'
unix_tups = get_nyse_date_tups(start, end, unix=True)

In [3]:
fixtime = lambda x: pd.to_datetime(x, unit='ms', utc=True).tz_convert('America/New_York')
datetime_tups = [(fixtime(open_), fixtime(close)) for open_, close in unix_tups]

In [4]:
datetime_tups[0], datetime_tups[-1]

((Timestamp('2018-10-11 09:30:00-0400', tz='America/New_York'),
  Timestamp('2018-10-11 16:00:00-0400', tz='America/New_York')),
 (Timestamp('2023-10-09 09:30:00-0400', tz='America/New_York'),
  Timestamp('2023-10-09 16:00:00-0400', tz='America/New_York')))

In [21]:
def format_unix(unix, new_tz='America/New_York', string=False):
    dt = pd.to_datetime(unix, unit='ms', utc=True).tz_convert(new_tz)
    if not string:
        return dt
    else:
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    
def format_unix_series(unix_series, new_tz='America/New_York', string=False):
    dt_series = pd.to_datetime(unix_series, unit='ms', utc=True).tz_convert(new_tz)
    return dt_series

## some data inspection

In [5]:
test = get_oxford_dfs(n=1) # get the data

In [6]:
# make a datetime column with EST datetimes
test['datetime'] = pd.to_datetime(test['t'], utc=True, unit='ms').dt.tz_convert('America/New_York')

In [7]:
test

,c,h,l,n,o,t,v,vw,ticker,datetime
0,226.50,226.50,226.50,1,226.50,1539256980000,100,226.5000,TMO,2018-10-11 07:23:00-04:00
1,227.00,227.00,227.00,2,227.00,1539259740000,150,226.8633,TMO,2018-10-11 08:09:00-04:00
2,227.00,227.00,227.00,4,227.00,1539259920000,209,226.9957,TMO,2018-10-11 08:12:00-04:00
3,227.00,227.00,227.00,4,227.00,1539261180000,250,226.8020,TMO,2018-10-11 08:33:00-04:00
4,228.50,228.50,228.50,4,228.50,1539261360000,214,228.4902,TMO,2018-10-11 08:36:00-04:00
...,...,...,...,...,...,...,...,...,...,...
469674,495.75,496.03,495.65,251,496.03,1696881360000,8608,495.8342,TMO,2023-10-09 15:56:00-04:00
469675,495.85,495.90,495.69,291,495.72,1696881420000,10163,495.7920,TMO,2023-10-09 15:57:00-04:00
469676,495.93,495.99,495.77,482,495.86,1696881480000,19078,495.8682,TMO,2023-10-09 15:58:00-04:00
469677,496.09,496.21,495.78,722,495.86,1696881540000,34031,496.0076,TMO,2023-10-09 15:59:00-04:00


In [8]:
# sort by inside/outside market hours
# this bool has to be NONINCLUSIVE of the end time else later the resampler will create a 7th interval just to hold trades exactly at 4pm
bool_ = (test['datetime'].dt.time >= pd.to_datetime('09:30:00').time()) & (test['datetime'].dt.time < pd.to_datetime('16:00:00').time())
market_hours = test[bool_].reset_index(drop=True)
outside_hours = test[~bool_].reset_index(drop=True)

In [192]:
just_10_16 = market_hours[market_hours["datetime"].dt.date == pd.Timestamp("2018-10-16").date()]
just_10_15 = market_hours[market_hours["datetime"].dt.date == pd.Timestamp("2018-10-15").date()]

In [191]:
just_10_16

,c,h,l,n,o,t,v,vw,ticker,datetime
1152,230.5000,230.700,230.07,24,230.7000,1539696600000,19580,230.6863,TMO,2018-10-16 09:30:00-04:00
1153,230.2350,230.680,229.97,44,230.0100,1539696660000,2975,230.2270,TMO,2018-10-16 09:31:00-04:00
1154,230.8055,230.900,230.52,63,230.5200,1539696720000,3164,230.7231,TMO,2018-10-16 09:32:00-04:00
1155,231.5600,231.560,230.80,121,230.9106,1539696780000,7208,231.2196,TMO,2018-10-16 09:33:00-04:00
1156,231.2000,231.440,231.14,100,231.3200,1539696840000,8928,231.2867,TMO,2018-10-16 09:34:00-04:00
...,...,...,...,...,...,...,...,...,...,...
1536,235.2700,235.420,235.27,208,235.3900,1539719760000,15452,235.3479,TMO,2018-10-16 15:56:00-04:00
1537,235.3600,235.410,235.26,202,235.2700,1539719820000,17225,235.3299,TMO,2018-10-16 15:57:00-04:00
1538,235.3000,235.370,235.13,294,235.3700,1539719880000,23652,235.2445,TMO,2018-10-16 15:58:00-04:00
1539,235.1800,235.385,235.11,577,235.2900,1539719940000,56946,235.2300,TMO,2018-10-16 15:59:00-04:00


In [193]:
just_10_15

,c,h,l,n,o,t,v,vw,ticker,datetime
777,229.97,229.970,229.9700,13,229.97,1539610200000,25069,229.9726,TMO,2018-10-15 09:30:00-04:00
778,230.02,230.020,230.0100,9,230.01,1539610260000,262,230.0706,TMO,2018-10-15 09:31:00-04:00
779,229.65,230.355,229.6500,32,230.02,1539610320000,1625,230.0132,TMO,2018-10-15 09:32:00-04:00
780,229.28,229.790,229.2032,24,229.60,1539610380000,2425,229.5470,TMO,2018-10-15 09:33:00-04:00
781,229.39,229.390,229.3900,50,229.39,1539610440000,1669,229.3873,TMO,2018-10-15 09:34:00-04:00
...,...,...,...,...,...,...,...,...,...,...
1147,228.66,228.890,228.6600,96,228.75,1539633300000,6832,228.7841,TMO,2018-10-15 15:55:00-04:00
1148,228.70,228.700,228.5400,111,228.63,1539633360000,6960,228.6045,TMO,2018-10-15 15:56:00-04:00
1149,228.66,228.740,228.5800,102,228.74,1539633420000,6008,228.6570,TMO,2018-10-15 15:57:00-04:00
1150,228.64,228.740,228.5750,125,228.67,1539633480000,8447,228.6585,TMO,2018-10-15 15:58:00-04:00


In [43]:
intervals_65min = [
    ('09:30:00', '10:35:00'),
    ('10:35:00', '11:40:00'),
    ('11:40:00', '12:45:00'),
    ('12:45:00', '13:50:00'),
    ('13:50:00', '14:55:00'),
    ('14:55:00', '16:00:00')
]

intervals_65min = [(pd.to_datetime(start).time(), pd.to_datetime(end).time()) for start, end in intervals_65min]

In [9]:
len(set(market_hours['datetime'].dt.date))

1256

In [11]:
aggs = market_hours.set_index('datetime').groupby(pd.Grouper(freq='D')).resample('65T', offset='9H30T', label='left')\
        .agg({'o': 'first', 'h': 'max', 'l': 'min', 'c': 'last', 'v': 'sum'})

In [12]:
aggs.index = aggs.index.droplevel(0)
aggs.reset_index(inplace=True)

In [17]:
aggs

,datetime,o,h,l,c,v
0,2018-10-11 09:30:00-04:00,230.950,233.4500,229.5828,231.4025,466332
1,2018-10-11 10:35:00-04:00,231.480,231.7300,227.4329,228.4800,339431
2,2018-10-11 11:40:00-04:00,228.510,232.1100,228.5100,231.4800,161132
3,2018-10-11 12:45:00-04:00,231.350,231.4250,228.9300,228.9800,179900
4,2018-10-11 13:50:00-04:00,228.870,229.8000,224.0900,226.8900,327924
...,...,...,...,...,...,...
7511,2023-10-09 10:35:00-04:00,491.700,492.6000,490.3898,492.3400,108373
7512,2023-10-09 11:40:00-04:00,492.565,494.6700,492.1300,492.7300,88954
7513,2023-10-09 12:45:00-04:00,492.585,494.6027,492.0300,494.6027,93585
7514,2023-10-09 13:50:00-04:00,494.515,497.3890,493.6900,497.1050,179167


In [13]:
len(aggs) / 1256

5.984076433121019

In [14]:
assert len(aggs) < 1256 * 6, f"Expected len of < (1256 unique days * 6 intervals or less), got {len(aggs)}" # 6 intervals per day (at least), 1256 trading days in 5 years

In [15]:
len(set(aggs['datetime'].dt.date))

1256

In [16]:
for day in aggs.groupby(aggs['datetime'].dt.date):
    if len(day[1]) != 6:
        print(day[0], len(day[1]))

2018-11-23 4
2018-12-24 4
2019-07-03 4
2019-11-29 4
2019-12-24 4
2020-11-27 4
2020-12-24 4
2021-11-26 4
2022-11-25 4
2023-07-03 4


In [212]:
test_day = aggs[aggs['datetime'].dt.date == pd.Timestamp('2018-11-23').date()]

In [213]:
test_day

,datetime,o,h,l,c,v
180,2018-11-23 09:30:00-05:00,230.3000,233.02,230.30,232.88,216825
181,2018-11-23 10:35:00-05:00,232.7900,235.38,232.67,234.60,135671
182,2018-11-23 11:40:00-05:00,234.5878,235.50,234.39,235.50,237049
183,2018-11-23 12:45:00-05:00,235.5200,235.72,234.94,235.16,100241


In [215]:
test_day_raw

,c,h,l,n,o,t,v,vw,ticker,datetime
11527,231.33,231.340,230.30,24,230.300,1542983400000,16560,230.3368,TMO,2018-11-23 09:30:00-05:00
11528,231.32,231.320,230.84,35,230.890,1542983460000,2399,231.0192,TMO,2018-11-23 09:31:00-05:00
11529,230.67,231.080,230.67,32,231.080,1542983520000,2012,230.9087,TMO,2018-11-23 09:32:00-05:00
11530,230.49,230.540,230.48,40,230.480,1542983580000,2364,230.5048,TMO,2018-11-23 09:33:00-05:00
11531,230.84,230.840,230.61,18,230.610,1542983640000,1024,230.8879,TMO,2018-11-23 09:34:00-05:00
...,...,...,...,...,...,...,...,...,...,...
11729,235.34,235.380,235.07,134,235.380,1542995700000,9104,235.2264,TMO,2018-11-23 12:55:00-05:00
11730,235.27,235.305,235.20,122,235.305,1542995760000,8146,235.2480,TMO,2018-11-23 12:56:00-05:00
11731,235.16,235.310,235.12,117,235.270,1542995820000,8541,235.2147,TMO,2018-11-23 12:57:00-05:00
11732,235.24,235.300,235.12,115,235.120,1542995880000,8176,235.2167,TMO,2018-11-23 12:58:00-05:00


In [214]:
test_day_raw = market_hours[market_hours['datetime'].dt.date == pd.Timestamp('2018-11-23').date()]